In [2]:
from keras.models import Sequential
from keras.layers import Dense
import numpy
numpy.random.seed(7)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
dataset = numpy.loadtxt("./data/dn.csv", delimiter=" ")

In [4]:
X = dataset[:,0:11]
Y = dataset[:,11]

In [43]:
model = Sequential()
model.add(Dense(15, input_dim=11, activation='relu'))
model.add(Dense(11, activation='relu'))
model.add(Dense(11, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [44]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [45]:
model.fit(X, Y, epochs=30, batch_size=10)

Epoch 1/30
30266/30266 [==============================] - 4s 139us/step - loss: 0.3358 - acc: 0.8865
Epoch 2/30
30266/30266 [==============================] - 4s 134us/step - loss: 0.2559 - acc: 0.9062
Epoch 3/30
30266/30266 [==============================] - 4s 134us/step - loss: 0.2401 - acc: 0.9115
Epoch 4/30
30266/30266 [==============================] - 4s 133us/step - loss: 0.2338 - acc: 0.9156
Epoch 5/30
30266/30266 [==============================] - 4s 132us/step - loss: 0.2283 - acc: 0.9159
Epoch 6/30
30266/30266 [==============================] - 4s 134us/step - loss: 0.2238 - acc: 0.9182
Epoch 7/30
30266/30266 [==============================] - 4s 133us/step - loss: 0.2216 - acc: 0.9178
Epoch 8/30
30266/30266 [==============================] - 4s 134us/step - loss: 0.2161 - acc: 0.9200
Epoch 9/30
30266/30266 [==============================] - 4s 135us/step - loss: 0.2119 - acc: 0.9222
Epoch 10/30
30266/30266 [==============================] - 4s 133us/step - loss: 0.2150 - a

In [46]:
scores = model.evaluate(X, Y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

30266/30266 [==============================] - 1s 19us/step

acc: 93.15%


In [38]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.preprocessing import sequence

In [69]:
model = Sequential()
model.add(Dense(15, input_dim=11, activation='relu'))
#model.add(LSTM(25, input_shape=(1,11)))
#model.add(Dense(1, activation='sigmoid'))

In [5]:
import tensorflow as tf
from tensorflow.contrib.rnn import BasicLSTMCell
from tensorflow.python.ops.rnn import dynamic_rnn

In [39]:
cats = 2
fsize = 1500
x = tf.placeholder(tf.float32, [None, fsize], name="X placeholder")
y = tf.placeholder(tf.float32, [None, cats])
seq_length = tf.placeholder(tf.int32)
keep_prob = tf.placeholder(tf.float32)
n_hidden = 25
n_classes = cats
learning_rate = 0.1
dropout = 0.75
weights = {
    'out' : tf.Variable(tf.random_normal([n_hidden, cats])),
    'out_bw' : tf.Variable(tf.random_normal([n_hidden, cats]))
}

biases = {
    'out' : tf.Variable(tf.random_normal([cats]))
}
number_of_layers = 1

ValueError: 'X placeholder' is not a valid scope name

In [7]:
def network(x, weightss, biases, dropout, seq_length):
    fc1 = tf.reshape(x, [1,-1,fsize])
    
    #tf.reset_default_graph()
    lstm_fw_cell = BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_fw_multicell = tf.contrib.rnn.MultiRNNCell([lstm_fw_cell] * number_of_layers, state_is_tuple=True)
    
    lstm_bw_cell = BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_bw_multicell = tf.contrib.rnn.MultiRNNCell([lstm_fw_cell] * number_of_layers, state_is_tuple=True)
    
    (output_fw, output_bw), (state_fw, state_bw) = tf.nn.bidirectional_dynamic_rnn(lstm_fw_multicell, lstm_bw_multicell, fc1, dtype='float32', sequence_length=tf.reshape(seq_length, [1]))
    
    outputs_fw = tf.reshape(output_fw, [seq_length, n_hidden])
    outputs_bw = tf.reshape(output_bw, [seq_length, n_hidden])
    
    out = tf.matmul(outputs_fw, weightss['out']) + tf.matmul(outputs_bw, weights['out_bw']) + biases['out']
    return out

In [9]:

#pred = network(x, weights, biases, keep_prob, seq_length)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [30]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.FtrlOptimizer(learning_rate=learning_rate, l2_regularization_strength=1.0).minimize(cost)

prediction=tf.argmax(pred,1)
correct_pred = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [35]:
print(prediction)

Tensor("ArgMax_11:0", shape=(?,), dtype=int64)


In [37]:
with tf.Session() as sess:
    result = sess.run(prediction)
    print(result)

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_2' with dtype int32
	 [[Node: Placeholder_2 = Placeholder[dtype=DT_INT32, shape=<unknown>, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder_2', defined at:
  File "/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/anaconda3/lib/python3.6/site-

In [27]:
print(pred)

Tensor("add_1:0", shape=(?, 2), dtype=float32)
